# **Manipuler une image grâce au code: partie 2**

## Introduction à l'analyse des images - 32M7132

*Adrien Jeanrenaud (adrien.jeanrenaud@unige.ch)*

<div class="alert alert-block alert-info">
<b>Manipulations partie 2</b> : poursuite de nos explorations des images en tant que matrice:
</div>

## **Plan du cours**

> **Corriger les images**
> * La luminosité
> * Le contraste

> **Histogramme: égliser les couleurs**
> * Qu'est-ce qu'un histogramme ?
> * Histogramme des images 2D
> * Histogramme des images 3D
> * Égaliser les hisogrammes 2D et 3D
> * Égaliser de manière adaptative

> **K-means: les couleurs principales**
> * Qu'est-ce qu'un "k-means"?
> * Appliquer l'algorithme
> * Récupérer les informations

In [ ]:
# importer les librairies nécessaires

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
# importer une image

image = " "
img = cv2.imread(image)

In [ ]:
# Visualiser

color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(color)
plt.show()

In [ ]:
# et en valeurs de gros

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray, cmap="gray")
plt.show()

## **1. Corriger les images**

### 1.1 L'exposition

Il est possible de corriger l'exposition, c'est à dire d'éclaircir ou d'assombrir une image. En utilisant le correction gamma, on contrôle la luminosité en changeant les rations RGB

In [ ]:
from skimage import exposure

?exposure.adjust_gamma

**En dessous de 1, l'image s'éclaircit, en dessus de 1 elle s'assombrit**

In [ ]:
gray_gamma = exposure.adjust_gamma(gray, gamma = 2.25)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_gamma,cmap = "gray")

**La correction fonctionne également sur des images couleurs**

In [ ]:
#image couleur
color_gamma = exposure.adjust_gamma(color, gamma = 2.25)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en couleurs')
ax1.imshow(color)

ax2.set_title ('Image transformée')
ax2.imshow(color_gamma)

<div class="alert alert-block alert-danger">
<b>L'outil est intéressant, enfin si l'on connait notre image et ses besoins.</b>
</div>

### 1.2 Le contraste

Le contraste définit la répartition de lumière dans l'image.
Modifier le contraste de l'image permet d'ouvrir la fenêtre des pixels ; si les valeurs min et max on peut d'écart, il est possible d'augmenter la rangée des valeurs utilisées

#### 1.2.1 Mofifier le contraste à la main

Modifier le contraste c'est appliquer une transformation point par point. On peut le faire à la main en changeant chaque point.

#### Modifier le contraste des images en valeurs de gris à la main

In [ ]:
# Trouver les valeurs min et max 

ma = gray.max()
mi = gray.min()
print(mi,ma)

In [ ]:
# Convertir l'image en float et ouvrir la fenêtre de valeurs

c = gray.astype(float)
gray_c = 255.0*(c-mi)/(ma-mi+0.0000001).astype(int)

In [ ]:
# Est-ce que ça a bien fonctionné ?

ma1 = gray_c.max()
mi1 = gray_c.min()
print(mi1,ma1)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_c,cmap = "gray")

#### Modifier le contraste des images en couleurs

In [ ]:
# On divise l'image en différents r, g, b 

r,g,b = cv2.split(img)

In [ ]:
ma = r.max()
mi = r.min()
print(mi,ma)
c = r.astype(float)
im1r = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
ma = g.max()
mi = g.min()
print(mi,ma)
c = g.astype(float)
im1g = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
ma = b.max()
mi = b.min()
print(mi,ma)
c = b.astype(float)
im1b = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
# On remet les canaux ensemble
# Attention à l'ordre des canaux de couleurs

color_c = cv2.merge([im1b, im1g, im1r]).astype(int)


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image couleur')
ax1.imshow(color)

ax2.set_title ('Image transformée')
ax2.imshow(color_c)

<div class="alert alert-block alert-danger">
<b>Ces deux opérations sont là pour montrer qu'il est possible de modifier les valeurs de la matrice point par point.</b>
</div>

## **2. Histogramme: égliser les couleurs**

### 2.1 Qu'est-ce qu'un histogramme ?

Un histogramme est une manière de représenter les couleurs et les valeurs de gris pour voir leur réparition au sein de l'image. Cela permet de voir si l'image contient quantitativement plus de blanc ou plus de bleu, par exemple. Cette manière de représenter les valeurs d'une image permet également, dans un second temps, d'y appliquer une égalisation afin d'avoir une meilleure répartition des valeurs de gris ou des couleurs.

### 2.2 Histogramme des images 2D

Quelle répartition entre noir(255) et blanc(255)?

In [ ]:
# On reprend notre image en valeurs de gris

gray.shape

In [ ]:
# 1ère manière: avec OpenCV
# cv2.calcHist(image, canal, masque, taille, liste des valeurs possibles)

gray_hist = cv2.calcHist([gray],[0],None,[256],[0,255])
plt.plot(gray_hist)
plt.title("Histogramme avec OpenCV")
plt.show()

In [ ]:
# 2e manière: avec Matplotlib et Numpy

plt.figure(figsize=(12, 8))
plt.hist(gray.ravel(),256,[0,255]) # ravel permet de passer d'une matrice en 2D à 1D
plt.title("Histogramme avec Matplotlib et Numpy")
plt.show()

### 2.3 Histogramme des images 3D

Pour faire l'histogramme d'une image en couleurs, il faut séparer les canaux ainsi que, pour chaque matrice, la réduire à 1D.

In [ ]:
# on reprend notre image couleur

color.shape

In [ ]:
# La fonction "ravel" permet de passer à 1D

color_ravel = color.ravel()

print("Dimensions de mon image couleur avant :  ", color.shape, 
      "\nDimensions de mon image couleur après la fonction ravel : ", color_ravel.shape)

In [ ]:
# Est-ce que mes deux images sont égales ?

color.shape[0]*color.shape[1]*color.shape[2] == color_ravel.shape[0]

In [ ]:
# Histogramme de l'image couleur
# On applique la fonction ravel à chacun des canaux de couleur

plt.figure(figsize=(12, 8))
plt.hist(color.ravel(),256,[0,256], color="Gray", label="Couleurs")
plt.hist(color[:,:,0].ravel(),256,[0,256], color="Red", label="Rouge")
plt.hist(color[:,:,1].ravel(),256,[0,256], color="Green", label="Vert")
plt.hist(color[:,:,2].ravel(),256,[0,256], color="Blue", label="Bleu")
plt.legend()
plt.title("Histogramme d'une image couleur")
plt.show()

In [ ]:
# Avec OpenCV


plt.figure(figsize=(12, 8))
hist = cv2.calcHist([color.ravel()],[0],None,[256],[0,256])
hist1 = cv2.calcHist([color],[0],None,[256],[0,256])
hist2 = cv2.calcHist([color],[1],None,[256],[0,256])
hist3 = cv2.calcHist([color],[2],None,[256],[0,256])
plt.plot(hist, color="Gray", label="Couleurs")
plt.plot(hist1, color="Red", label="Rouge")
plt.plot(hist2, color="Green", label="Vert")
plt.plot(hist3, color="Blue", label="Bleu")
plt.title("Histogramme avec OpenCV")
plt.legend()
plt.show()

### 2.4 Égaliser les hisogrammes 2D et 3D

Egaliser un histogramme permet d'affiner le contraste d'une image en distribuant mieux les valeurs, de cette manière l'intensité est mieux répartie.

In [ ]:
# Histogramme de base en valeurs de gris

plt.figure(figsize=(12, 8))
gray_hist = cv2.calcHist([gray],[0],None,[256],[0,255])
plt.plot(gray_hist)
plt.title("Histogramme avec OpenCV")
plt.show()

In [ ]:
?cv2.equalizeHist

In [ ]:
# Histogramme égalisé

plt.figure(figsize=(12, 8))
gray_equ = cv2.equalizeHist(gray)
gray_equ_hist = cv2.calcHist([gray_equ],[0],None,[256],[0,255])
plt.plot(gray_equ_hist)
plt.title("Histogramme égalisé")
plt.show()

In [ ]:
# Comparaison des histogrammes

plt.figure(figsize=(12, 8))
gray_hist = cv2.calcHist([gray],[0],None,[256],[0,255])
gray_equ = cv2.equalizeHist(gray)
gray_equ_hist = cv2.calcHist([gray_equ],[0],None,[256],[0,255])
plt.plot(gray_hist, color="Gray", label="Image de base")
plt.plot(gray_equ_hist, color="Orange", label="Image égalisée")
plt.title("Comparaison des histogrammes")
plt.legend()
plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray, cmap="gray")

ax2.set_title ('Image égalisée')
ax2.imshow(gray_equ, cmap="gray")

In [ ]:
# Cela fonctionne également avec les couleurs

R, G, B = cv2.split(color)

R_equ = cv2.equalizeHist(R)
G_equ = cv2.equalizeHist(G)
B_equ = cv2.equalizeHist(B)
color_equ = cv2.merge((R_equ, G_equ, B_equ))

plt.figure(figsize=(12, 8))
hist = cv2.calcHist([color.ravel()],[0],None,[256],[0,255])
hist_equ = cv2.calcHist([color_equ.ravel()],[0],None,[256],[0,255])
plt.plot(hist, color="Gray", label="Image de base")
plt.plot(hist_equ, color="Orange", label="Image égalisée")
plt.title("Comparaison des histogrammes")
plt.legend()
plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en couleurs')
ax1.imshow(color)

ax2.set_title ('Image égalisée')
ax2.imshow(color_equ)

### 2.5 Égaliser de manière adaptative

L'égalisation adaptative prend en compte les différentes régions de l'image en adaptant localement l'égalisation.

In [ ]:
?exposure.equalize_adapthist

In [ ]:
# Avec une image en valeurs de gris
# Plus le clip_limit est haut, plus il y a de contraste (entre 0 et 1)

gray_hist_adapt = exposure.equalize_adapthist(gray, kernel_size=None, clip_limit=0.8, nbins=256)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray, cmap="gray")

ax2.set_title ('Image égalisée de manière adaptative')
ax2.imshow(gray_hist_adapt, cmap="gray")

In [ ]:
# Comparaison entre égalisation adaptative de l'histogramme et égalisation de l'histogramme

## Valeurs de gris

# histogramme en valeurs de gris
gray_calc_hist = exposure.histogram(gray, nbins=256) # calcul de l'histogramme

# égalisation adaptative de l'histogramme en valeurs de gris
gray_histEA = exposure.equalize_adapthist(gray, kernel_size=None, clip_limit=0.1, nbins=256)
gray_calc_histEA = exposure.histogram(gray_histEA, nbins=256) # calcul de l'histogramme

# égalisation de l'histogramme en valeurs de gris
gray_histE = exposure.equalize_hist(gray, nbins=256)
gray_calc_histE = exposure.histogram(gray_histE, nbins=256) #calcul de l'histogramme




In [ ]:
# Visualiser l'histogramme en valeurs de gris et les deux égalisations 

plt.figure(figsize=(12, 8))
plt.plot(gray_calc_hist[0], color="Gray", label="Image de base")
plt.plot(gray_calc_histEA[0], color="Red", label="Image égalisée de manière adaptative")
plt.plot(gray_calc_histE[0], color="Orange", label="Image égalisée")
plt.title("Comparaison des histogrammes en valeurs de gris")
plt.legend()
plt.show()

In [ ]:
#image couleur

color_hist_equ = exposure.equalize_adapthist(color, kernel_size=None, clip_limit=1)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en couleurs')
ax1.imshow(color)

ax2.set_title ('Image égalisée de manière adaptative')
ax2.imshow(color_hist_equ)

## **3. K-means: les couleurs principales**


### 3.1 Qu'est-ce qu'un "k-means"?

K-means, ou k-moyenne en français, est une méthode pour séparer ses données, en choisissant le nombre de regroupements (cluster) à avoir au final (k).
C'est un alogrithme non-supervisé, c'est-à-dire qu'il ne sait pas à quelle classe appartient chacun des points qu'on lui donne. Il va donc attribuer une classe à chacun de nos points.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fproxy%2F0*G7LC_oXt4mNzavMe.jpg&f=1&nofb=1&ipt=a7daeb356e10138e7791820e5e1eddf8b9f42cebcf851eb34e8e9f50e9b130b9&ipo=images" title="k-means"/>

**L'algorithme s'applique à divers types de données, dont les images** 

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Failephant.com%2Fwp-content%2Fuploads%2F2020%2F08%2Fdominant-colors-kmeans.jpg&f=1&nofb=1&ipt=1f1296a10a4aad084e690da2781ff4b1bcf2e78b83041ea69714938b85900beb&ipo=images" title="k-means2"/>

In [ ]:
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

### 3.2 Appliquer l'algorithme

In [ ]:
# on reprend son image couleur

plt.imshow(color)
plt.show()

In [ ]:
# d'abord il faut passer d'une image 3D à une image 2D
(h,w,c) = color.shape
img2D = color.reshape(h*w,c)
print(img2D)
print(img2D.shape)

In [ ]:
# On choisit le nombre de regroupements (clusters) à l'aide du k

n = 6
kmeans_model = KMeans(n_clusters=n)

In [ ]:
# On applique l'algorithme à notre image

s=kmeans_model.fit(img2D)
cluster_labels = kmeans_model.fit_predict(img2D)

In [ ]:
# On peut voir combien de pixels il y a par regroupemets

from collections import Counter
labels_count = Counter(cluster_labels)
print(labels_count)

In [ ]:
# On peut voir quels sont les centres des regroupements, donc les couleurs

rgb_cols = kmeans_model.cluster_centers_.round(0).astype(int)

print(rgb_cols)

In [ ]:
# L'image est désormais faite de "k" couleurs
img_quant = np.reshape(rgb_cols[cluster_labels],(h,w,c))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,12))
ax[0].imshow(color)
ax[0].set_title('Image en couleur')
ax[1].imshow(img_quant)
ax[1].set_title(f'Image avec un k-means = {n}')

<div class="alert alert-block alert-danger">
<b>Cette méthode à l'intérêt de donner des informations concises sur les couleurs d'une image. Toutefois la méthode a ses défaut. Principalement au niveau du choix du nombre de regroupements, même s'il existe des outils pour s'y retrouver</b>.
</div>

### 3.3 Récupérer les informations

In [ ]:
per = [] # ajout
colors = [] # ajout
percentages = (np.unique(kmeans_model.labels_,return_counts=True)[1])/img2D.shape[0]
p_and_c = zip(percentages,rgb_cols)
p_and_c = sorted(p_and_c,reverse=True)

block = np.ones((50,50,3),dtype='uint')
plt.figure(figsize=(12,8))
for i in range(n):
    per.append((round(p_and_c[i][0]*100,2))) # ajout
    colors.append((p_and_c[i][1]).tolist()) # ajout
    plt.subplot(1,n,i+1)
    block[:] = p_and_c[i][1]
    plt.imshow(block)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(str(round(p_and_c[i][0]*100,2))+'%')


bar = np.ones((50,500,3),dtype='uint')
plt.figure(figsize=(12,8))
plt.title(f'Proportions of colors in the image (k-means={n})')
start = 0
i = 1
for p,c in p_and_c:
    end = start+int(p*bar.shape[1])
    if i==n:
        bar[:,start:] = c[::1]
    else:
        bar[:,start:end] = c[::1]
    start = end
    i+=1

#plt.savefig(" ")
plt.imshow(bar)

In [ ]:
# on peut également récupérer les informations sous type de tableur

infos = {
      "nom": image,
      "couleurs": colors,
      "pourcentages": per
  }

In [ ]:
infos

In [ ]:
# transformer en Dataframe

import pandas as pd
df = pd.DataFrame.from_dict(infos, orient="index").T
df

<div class="alert alert-block alert-warning">
<b>Exercice pour le prochain cours</b>: à partir d'une image couleur, vous devez appliquer les différentes transformations vues pendant le cours. Lorsque vous aurez vos quatres images (exposition, contraste, égalisation et égalisation adaptative) appliquez l'algorithme k-means et commentez les différences entre les couleurs dominantes.</div>